In [1]:
import pandas as pd

In [2]:
data =  pd.read_csv("channel_data.csv")
data.head()

,title,description,url
0,Rasam Vada Recipe | Tomato Rasam | Perfect Vad...,Rasam Powder: https://youtu.be/YIWBthqArCM?t=5...,https://www.youtube.com/watch?v=7LpEu6rTyHs
1,Peri Peri Chicken with Butter Rice | Spicy Hom...,Full written recipe of Chicken in Peri Peri Sa...,https://www.youtube.com/watch?v=d9_YSS1lB1w
2,Homestyle Turai Ki Sabzi | Turai Ka Bharta | त...,Full written recipe of Turai Ki Sabzi\nPrep ti...,https://www.youtube.com/watch?v=nX5vogjw_yk
3,Chicken Pepper Fry Recipe | Pepper Chicken Mas...,Full written recipe for Chicken Pepper Fry\nPr...,https://www.youtube.com/watch?v=5R6nNJC-FVE
4,Achari Paneer Recipe | Dhaba Style Paneer Grav...,Full written recipe of Achari Paneer \nPrep ti...,https://www.youtube.com/watch?v=P514UYjzZ-E


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1457 entries, 0 to 1456
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   title        1457 non-null   object
 1   description  1380 non-null   object
 2   url          1457 non-null   object
dtypes: object(3)
memory usage: 34.3+ KB


In [4]:
#fillings na values with empty string
data["description"] = data["description"].fillna("")

## LLM parser

For eg. :
The first description is 
```
data['description'][0] = "Rasam Powder: https://youtu.be/YIWBthqArCM?t=59&si=zpnugmEpzvQkN5mb\n\nFull written recipe Rasam Vada\n\nPrep time: 15-20 minutes (excluding soaking time)\nCooking time: 25-30 minutes\nServes: 4-5 people\n\nIngredients:\nRasam\nTUR DAL | तूर दाल ¼ CUP (SOAKED)\nWATER | पानी AS REQUIRED \nTURMERIC POWDER | हल्दी पाउडर ¼ TSP\nOIL | तेल 2 TBSP\nMUSTARD SEED | राई ½ TSP\nCURRY LEAVES | कड़ी पत्ता 12-15 NOS. \nGREEN CHILLI | हरी मिर्च 2 NOS. (SLIT)\nRED CHILLI | लाल मिर्च 4 NOS. \nGARLIC | लहसुन 6 CLOVES (ROUGHLY CHOPPED)\nASAFOETIDA | हींग ¼ TSP\nTOMATO | टमाटर 2 NOS. (CHOPPED)\nCORIANDER STEMS | धनिए के डंठल A HANDFUL (CHOPPED)\nSALT | नमक TO TASTE\nTURMERIC POWDER | हल्दी पाउडर ¼ TSP\nTAMARIND | इमली A LARGE LEMON SIZED BALL. (SOAKED)\nJAGGERY | गुड़ 1 TBSP\nHOT WATER | गरम पानी AS REQUIRED \nRASAM POWDER | रसम पाउडर 3 TBSP\nBLACK PEPPER | काली मिर्च A LARGE PINCH (FRESHLY CRUSHED)\nFRESH CORIANDER | हरा धनिया A HANDFUL (ROUGHLY CHOPPED)\n\nVada\nURAD DAL | उरद दाल 1 CUP\nCOLD WATER | ठंडा पानी AS REQUIRED \nSALT | नमक TO TASTE\nCURRY LEAVES | कड़ी पत्ता 10-12 NOS. (CHOPPED)\nGREEN CHILLI | हरी मिर्च 2 NOS. (CHOPPED)\nGINGER | अदरक 1 INCH (CHOPPED)\nBLACK PEPPER | काली मिर्च A PINCH (FRESHLY CRUSHED)\nASAFOETIDA | हींग ¼ TSP\n Method:\nAdd the washed & soaked tur dal in a pressure cooker along with enough water & turmeric powder, put the lid & pressure cook the dal for 3-4 whistles over high flame.\nOnce cooked let the cooker depressurize naturally & then take off the lid, whisk the dal well using a whisk, make sure it should become completely smooth & watery.\nSet a stock pot over high heat & add oil into it, once the oil gets hot add in the red chillies, mustard seeds, curry leaves, green chilli & garlic, stir well & cook briefly.\nFurther add asafoetida, tomato, salt & coriander stem, stir well & cook until the tomatoes get mushy.\nAdd turmeric powder & tamarind pulp along with jaggery & hot water, stir & cook for 3-4 minutes until the raw smell of the tamarind is gone.\nThen add the cooked tur dal & hot water to adjust the consistency, rasam is always watery, it is not as thick as sambar.\nLet the rasam simmer for 7-8 minutes, if it thickens up then add more hot water to it.\nOnce you have let the rasam simmer, add rasam powder to a bowl along with a few tablespoons of water, stir well until no lumps remain & add the mixture to the rasam & stir well.\nTaste & adjust the salt at this stage & add crushed black peppercorns along with fresh coriander, stir & cook for 2-3 minutes, the rasam is ready. \nWash the urad dal & soak it for 3-4 hours, once the dal is soaked, drain the excess water & add the soaked urad dal into a mixer grinder jar & grind the dal coarsely  using minimal water (it should be ice cold), the texture of the dal should neither be completely smooth nor extremely coarse.\nFurther transfer the dal into a bowl & whisk it for 3-4 minutes using your hand or a whisk until it gets light & fluffy.\nTo check if the batter is ready, add a small portion of it in a bowl of water, if the batter floats, then you can continue with the addition of further ingredients.\nFurther add salt, ginger, green chilli, asafoetida, curry leaves & mix it very gently.\nTo fry the vadas, heat the oil in a kadhai until it gets moderately hot or 170 C.\nTo shape the vadas dip your hand in cold water & scoop out a small bite sized portion of the batter from the bowl & carefully drop it into the hot oil, add other portions similar & then immediately spash hot oil over the vadas so that they puff up & then fry over medium flame while flipping it at regular intervals until light golden brown.\nOnce the vadas get golden brown & crisp, remove them using a spider & transfer them into a sieve, the vadas are ready.\nTo serve the rasam vada, add a few vadas in a bowl & pour hot rasam over it, you can also directly add the vadas in the rasam after frying. \n\n\n#YFL #SanjyotKeer #RasamVada\n\nThe Music I use in All my videos - https://bit.ly/3CdPbSc\n(Best for independent creators)\n\nFollow us on all platforms: \nFacebook - https://bit.ly/3o6alLt\nInstagram - https://bit.ly/2RFkL8z\n\nFollow my personal handles here: (Chef Sanjyot Keer) \nFacebook - https://bit.ly/3tpjqQA\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\nInstagram - https://bit.ly/3o0FZK8\nTwitter  -  https://bit.ly/33qQpJN\n\nIntro 0:00\nRasam 1:31\nVada 6:50\nPlating 12:50\nOutro 13:55"

```
we can extract things like 
recipe_name, ingredents, preparation_time, cooking_time by passing this description to a LLM model and asking it to return it into JSON format which is then parsed, formatted and validated through pydantic


In [40]:
import ollama
response = ollama.chat(model='llama3', messages=[
  {
    'role': 'user',
    'content': 'Why is the sky blue?',
  },
])
print(response['message']['content'])

The sky appears blue because of a phenomenon called scattering, where shorter (blue) wavelengths of light are dispersed throughout the atmosphere more than longer (red) wavelengths. Here's a simplified explanation:

1. **Sunlight**: When sunlight enters Earth's atmosphere, it contains all the colors of the visible spectrum, including red, orange, yellow, green, blue, and violet.
2. **Scattering**: As sunlight travels through the atmosphere, it encounters tiny molecules of gases like nitrogen (N2) and oxygen (O2), as well as larger particles like dust, pollen, and water droplets. These particles scatter the light in all directions.
3. **Blue light dominance**: The smaller molecules and particles are more effective at scattering shorter wavelengths of light, such as blue and violet. This is because they have a stronger interaction with these wavelengths due to their size and energy.
4. **Red light absorption**: Longer wavelengths of light, like red and orange, tend to travel in straighte

In [17]:
import pandas as pd
from langchain.prompts import PromptTemplate
from langchain.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field
from typing import List
import google.generativeai as genai
from api_keys import GEMINI_API_KEY

genai.configure(api_key=GEMINI_API_KEY)
model = genai.GenerativeModel('gemini-1.5-flash')


class RecipeInfo(BaseModel):
    recipe_name: str = Field(description="Name of the recipe (in English), concise like Dal bati, Red Curry, Rajma Chawal, Rasam wada")
    ingredents: dict = Field(description="Combined single key:value based dict of vegetables,pulses,meat used with quantities like {{'TOMATOES':4,'CHICKEN':'500g'}} ")
    preparation_time: int = Field(description="Preparation time in minutes like 15, 25, 40")
    cooking_time: int = Field(description="Cooking time in minutes like 10, 45, 1hr should be 60")

parser = PydanticOutputParser(pydantic_object=RecipeInfo)
prompt_template = PromptTemplate(
    template=
    """Extract the following information in JSON formal: \n{format_instructions}\n\n
    from the below text: {text}. \n\n
    Links are not required to be extracted. \n\n Return in JSON object only.
    """,
    input_variables=["text"],
    partial_variables={"format_instructions": parser.get_format_instructions()}
)


def query_gemini(prompt):
    try:
        response = model.generate_content(prompt)
        return response.text
    except Exception as e:
        print(f"Error calling Gemini API: {e}")
        return None  # Or return a specific error message


def process_recipe(text):
    # Use Gemini instead of Ollama
    prompt = prompt_template.format(text=text)
    response = query_gemini(prompt)
    # print(response)
    if response:
        try:
            parsed_response = parser.parse(response)
            return parsed_response
        except Exception as e:
            print(f"Error parsing response: {e}")
            return None  # Or return a specific error message
    else:
        return None  # Handle no response from Gemini


In [7]:
test_sample_str = data["description"][1]
x = process_recipe(test_sample_str)

```json
{
  "recipe_name": "Chicken in Peri Peri Sauce with Butter Rice",
  "ingredents": {
    "KASHMIRI RED CHILLI POWDER": "1 TBSP",
    "SPICY RED CHILLI POWDER": "1 TSP",
    "GARLIC POWDER": "1 TSP",
    "ONION": "1 TSP",
    "DRY MANGO POWDER": "1 TSP",
    "BLACK PEPPER POWDER": "1/2 TSP",
    "OREGANO": "1 TSP",
    "SUGAR": "1 TSP",
    "SALT": "1 TSP",
    "CHICKEN": "400-450 GRAMS (BREAST)",
    "OLIVE OIL": "1 TBSP",
    "GINGER GARLIC GREEN CHILLI PASTE": "1 TBSP",
    "PERI PERI SEASONING": "2 TBSP",
    "VINEGAR": "1 TSP",
    "RED BELL PEPPER": "1 NO.",
    "TOMATO": "2 NOS.",
    "GREEN CHILLI": "3 NOS.",
    "LEMON ZEST": "OF 1 LEMON.",
    "LEMON JUICE": "1/2 A LEMON",
    "FRESH CREAM": "2-3 TBSP",
    "FRESH CORIANDER": "A HANDFUL (CHOPPED)",
    "BUTTER": "2 TBSP",
    "BASMATI RICE": "4 CUPS (COOKED)"
  },
  "preparation_time": 15,
  "cooking_time": 30
}
```


In [9]:
print(x.recipe_name)
print(x.ingredents)
print(x.preparation_time)
print(x.cooking_time)

Chicken in Peri Peri Sauce with Butter Rice
{'KASHMIRI RED CHILLI POWDER': '1 TBSP', 'SPICY RED CHILLI POWDER': '1 TSP', 'GARLIC POWDER': '1 TSP', 'ONION': '1 TSP', 'DRY MANGO POWDER': '1 TSP', 'BLACK PEPPER POWDER': '1/2 TSP', 'OREGANO': '1 TSP', 'SUGAR': '1 TSP', 'SALT': '1 TSP', 'CHICKEN': '400-450 GRAMS (BREAST)', 'OLIVE OIL': '1 TBSP', 'GINGER GARLIC GREEN CHILLI PASTE': '1 TBSP', 'PERI PERI SEASONING': '2 TBSP', 'VINEGAR': '1 TSP', 'RED BELL PEPPER': '1 NO.', 'TOMATO': '2 NOS.', 'GREEN CHILLI': '3 NOS.', 'LEMON ZEST': 'OF 1 LEMON.', 'LEMON JUICE': '1/2 A LEMON', 'FRESH CREAM': '2-3 TBSP', 'FRESH CORIANDER': 'A HANDFUL (CHOPPED)', 'BUTTER': '2 TBSP', 'BASMATI RICE': '4 CUPS (COOKED)'}
15
30


In [11]:
new_cols = ["recipe_name", "ingredients", "preparation_time", "cooking_time"]

def extract_recipe_info(text):
    parsed_data = process_recipe(text)
    if parsed_data:
        return [parsed_data.recipe_name, [parsed_data.ingredents], parsed_data.preparation_time, parsed_data.cooking_time]
    else:
        return [None] * len(new_cols)  # Return a list of None for all columns if parsing fails

ValueError: Column description must have a numeric dtype. Found 'object' instead

In [31]:
df = data[:5]


for col in new_cols:
    df[col] = None

for i,row in  df.iterrows():
    if df.loc[i, "recipe_name"] is None:
        x = extract_recipe_info(row["description"])
        for j, col in enumerate(new_cols):
            df.loc[i, j] = x[j]
    
    


/var/folders/kx/r7rzvccn703dmc_gt7w0lrbr0000gn/T/ipykernel_61264/2083699658.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col] = None
/var/folders/kx/r7rzvccn703dmc_gt7w0lrbr0000gn/T/ipykernel_61264/2083699658.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col] = None
/var/folders/kx/r7rzvccn703dmc_gt7w0lrbr0000gn/T/ipykernel_61264/2083699658.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value inst

In [37]:
df = pd.read_csv("channel_data_parsed_g.csv")
df[20:30]

,title,description,url,recipe_name,ingredients,preparation_time,cooking_time
20,Gujarati Rasse Waale Aloo | Kanda Tameta Batet...,Full written recipe of Kanda Bateta Tameta nu ...,https://www.youtube.com/watch?v=uMLX6Dt1whk,Kanda Bateta Tameta nu Shaak / Gujarati Rasse ...,"{'POTATO': 4, 'TOMATO': 2, 'ONION': 1, 'GROUND...",10.0,15.0
21,Raw Mango Rasam Recipe | Summer Special Recipe...,Rasam Powder & Tomato Rasam: https://youtu.be/...,https://www.youtube.com/watch?v=WsVIpfe8eDY,Raw Mango Rasam,"{'RAW MANGO': 1, 'GREEN CHILLI': 3, 'WATER': '...",5.0,25.0
22,Egg Dum Biryani for Beginners | Quick Bachelor...,Full written recipe for Bachelor’s Egg Biryani...,https://www.youtube.com/watch?v=StgQNDMXHIs,Bachelor’s Egg Biryani,"{'BASMATI RICE': 1, 'WATER': 0, 'ONION': 3, 'G...",10.0,30.0
23,Punjabi Tari Waale Chane | Mom's Special Recip...,Tari Waale Chane \nPrep time: 5-10 mins (exclu...,https://www.youtube.com/watch?v=B8CT6-D2gOM,Tari Waale Chane,"{'BLACK CHICKPEAS': 1, 'ONION': 3, 'GARLIC': 1...",10.0,25.0
24,Eggless Churros Recipe | Perfect & Easy Churro...,Full written recipe of Eggless Churros\n\nPrep...,https://www.youtube.com/watch?v=fc-pm4asWm0,Eggless Churros,"{'CASTER SUGAR': 0.25, 'CINNAMON POWDER': 0.5,...",10.0,25.0
25,Lauki Chana Dal Recipe | Home-Style Cooker Sab...,Lauki ki Sabzi: https://youtu.be/emj4HI3p8Uk?s...,https://www.youtube.com/watch?v=NnUyvIXmkgc,Lauki Chana Dal,"{'CHANA DAL': '3/4 CUP', 'GHEE': '2 TBSP', 'CU...",5.0,15.0
26,Veg Soya Matar Pulao | Cooker Pulao Recipe | क...,Full written recipe of Soya Matar Pulao\nPrep ...,https://www.youtube.com/watch?v=mkqQJl9USGM,NaN,NaN,NaN,NaN
27,Gatte Ki Sabzi | No Onion No Garlic | बिना प्य...,Dhaba Style Gatte Ki Sabzi: https://youtu.be/4...,https://www.youtube.com/watch?v=X-6i1Y7Pmys,NaN,NaN,NaN,NaN
28,Karela Aloo Ki Sabzi | Less Bitter Karela Tips...,Full written recipe for Karela Aloo\nPrep time...,https://www.youtube.com/watch?v=s0rGqmW2A5o,NaN,NaN,NaN,NaN
29,Bengali Tok Dal Recipe | Quick Raw Mango Dal |...,Full written recipe of Bengali Tok Dal \n\nPre...,https://www.youtube.com/watch?v=Fy1yZuhmSIo,NaN,NaN,NaN,NaN


In [38]:
df[['recipe_name', 'ingredients', 'preparation_time', 'cooking_time']].head()

,recipe_name,ingredients,preparation_time,cooking_time
0,Rasam Vada,"{'TUR DAL': '0.25', 'WATER': 'AS REQUIRED', 'T...",20.0,30.0
1,Chicken in Peri Peri Sauce with Butter Rice,"{'CHICKEN': '400-450 GRAMS', 'OLIVE OIL': '3 T...",15.0,30.0
2,Turai Ki Sabzi,"{'SPONGE GOURD': 1, 'ONION': 2, 'TOMATO': 2, '...",10.0,25.0
3,Chicken Pepper Fry,"{'BLACK PEPPERCORNS': 2, 'CUMIN SEEDS': 1, 'FE...",10.0,20.0
4,Achari Paneer,"{'FENNEL SEEDS': 2, 'CUMIN SEEDS': 1, 'MUSTARD...",25.0,45.0
